**Dataset:**
personality type data (Lab 2 - Personality Profile Type.csv)

**Objective:**
classify Personality type as one of the following **using RNNs**. <br>
['INFJ', 'ENTP', 'INTP', 'INTJ', 'ENTJ', 'ENFJ', 'INFP', 'ENFP',
       'ISFP', 'ISTP', 'ISFJ', 'ISTJ', 'ESTP', 'ESFP', 'ESTJ', 'ESFJ']

**Evaluation metric:**
Precision

### Import used libraries

In [ ]:
!pip install contractions

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.7/110.7 kB 14.6 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
import string
import re

from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, classification_report

import contractions
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences

import tensorflow as tf
import spacy

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_colwidth', 500)

### Load Dataset

In [ ]:
df = pd.read_csv("/content/Lab 2 - Personality Profile Type.csv")
df.head(10)

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments https://www.youtube.com/watch?v=iz7lE1g4XM4 sportscenter not top ten plays https://www.youtube.com/watch?v=uCdfze1etec pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8 http://www.youtube.com/watch?v=u8ejam5DP3E On repeat for most of today.|||May the PerC Experience immerse you.|||The last ...
1,ENTP,'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin* That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This...
2,INTP,"'Good one _____ https://www.youtube.com/watch?v=fHiGbolFFGw|||Of course, to which I say I know; that's my blessing and my curse.|||Does being absolutely positive that you and your best friend could be an amazing couple count? If so, than yes. Or it's more I could be madly in love in case I reconciled my feelings (which at...|||No, I didn't; thank you for a link!|||So-called Ti-Si loop (and it can stem from any current topic/obsession) can be deadly. It's like when you're stuck in your o..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the other day. Esoteric gabbing about the nature of the universe and the idea that every rule and social code being arbitrary constructs created...|||Dear ENTJ sub, Long time no see. Sincerely, Alpha|||None of them. All other types hurt in deep existential ways that I want no part of.|||Probably a sliding scale that depends on individual preferences, like everything in humanity.|||Draco Malfoy also. I'd say he's either 358 or 368.|||I'm either 358..."
4,ENTJ,'You're fired.|||That's another silly misconception. That approaching is logically is going to be the key to unlocking whatever it is you think you are entitled to. Nobody wants to be approached with BS...|||But guys... he REALLY wants to go on a super-duper-long-ass vacation. C'mon guys. His boss just doesn't listen or get it. He even approached him logically and everything.|||Never mind. Just go on permanent vacation.|||Two months? I wouldn't be crazy about the idea. If you are really hi...
5,INTJ,"'18/37 @.@|||Science is not perfect. No scientist claims that it is, or that scientific information will not be revised as we discover new things. Rational thinking has been very useful to our society....|||INFP- Edgar Allen Poe was an INFP and he's in your siggy.|||People see the obvious Fi and are quick to put her as INFP. I agree that she has no Ne. I see her as an ISFP. Compare her to Haku (definite INFP). She is flat through most of Naruto.. but I don't...|||Lets get this party star..."
6,INFJ,"'No, I can't draw on my own nails (haha). Those were done by professionals on my nails. And yes, those are all gel. You mean those you posted were done by yourself on your own nails? Awesome!|||Probably the Electronic Screen Syndrome. With the advent of technology and social media, we all suffer from overstimulation on a daily basis. I'm guilty as well. In the past, I can be happy just...|||I love nail arts too! These are some of mine: 718282 718290 718298 718306 718314|||This is the first..."
7,INTJ,"'I tend to build up a collection of things on my desktop that i use frequently and then move them into a folder called 'Everything' from there it get sorted into type and sub type|||i ike to collect odd objects, even at work...a lot of people would call it junk but i like to collect it. Old unused software? ill take that off your hands :) i have a bunch of old adobe...|||i think its quite normal, i tend to only see my friends in real life every c

In [ ]:
df

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments https://www.youtube.com/watch?v=iz7lE1g4XM4 sportscenter not top ten plays https://www.youtube.com/watch?v=uCdfze1etec pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8 http://www.youtube.com/watch?v=u8ejam5DP3E On repeat for most of today.|||May the PerC Experience immerse you.|||The last ...
1,ENTP,'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin* That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This...
2,INTP,"'Good one _____ https://www.youtube.com/watch?v=fHiGbolFFGw|||Of course, to which I say I know; that's my blessing and my curse.|||Does being absolutely positive that you and your best friend could be an amazing couple count? If so, than yes. Or it's more I could be madly in love in case I reconciled my feelings (which at...|||No, I didn't; thank you for a link!|||So-called Ti-Si loop (and it can stem from any current topic/obsession) can be deadly. It's like when you're stuck in your o..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the other day. Esoteric gabbing about the nature of the universe and the idea that every rule and social code being arbitrary constructs created...|||Dear ENTJ sub, Long time no see. Sincerely, Alpha|||None of them. All other types hurt in deep existential ways that I want no part of.|||Probably a sliding scale that depends on individual preferences, like everything in humanity.|||Draco Malfoy also. I'd say he's either 358 or 368.|||I'm either 358..."
4,ENTJ,'You're fired.|||That's another silly misconception. That approaching is logically is going to be the key to unlocking whatever it is you think you are entitled to. Nobody wants to be approached with BS...|||But guys... he REALLY wants to go on a super-duper-long-ass vacation. C'mon guys. His boss just doesn't listen or get it. He even approached him logically and everything.|||Never mind. Just go on permanent vacation.|||Two months? I wouldn't be crazy about the idea. If you are really hi...
...,...,...
8670,ISFP,"'https://www.youtube.com/watch?v=t8edHB_h908|||IxFP just because I always think of cats as Fi doms for some reason. https://www.youtube.com/watch?v=ib28iGc9go4|||Especially on websites that have become a haven for neo nazis, such as PerC.|||I'm such a nerd. I've been learning Dutch on Duolingo and I'm having so much fun :)|||Duolingo is the shit! Oh my god I love this XD|||Right wingers do this because they lack political consciousness.|||I doubt it, but if they are real then I hope to he..."
8671,ENFP,"'So...if this thread already exists someplace else (which it does: http://personalitycafe.com/enfp-forum-inspirers/116902-movies-loved-enfp.html#post2952851), how the heck to I delete this one here???|||Ooops...I guess I didn't look too hard because I just started a Movies every ENFP should watch thread...|||So, this came up in the You Know You're An ENFP When... thread. (@Pucca : here we go! IT IS DONE :D) This isn't a list of films with ENFPs IN them, but rather ones that you believe E..."
8672,INTP,'So many questions when i do these things. I would take the purple pill. Pick winning lottery numbers. Do whatever I want.|||I worked in a brewery for a couple years here. I dont know if you're familiar with the local micros/craft at all but Mill st. was the name of it. Their tankhouse is my favorite of anything local.|||I am going to summon the great and powerful mr cafebot and his song of destruction. The four horseman ride. 

In [ ]:
df.columns

Index(['type', 'posts'], dtype='object')

In [ ]:
# Keep only the first two columns
dff = df.iloc[:, :2]

# Strip leading and trailing whitespaces from column names and then rename the columns
dff = dff.rename(columns=lambda x: x.strip())

# Rename the columns
dff = dff.rename(columns={"Unnamed: 0": "label",
                          "then I pick that one. + Super Troopers & Beerfest|||Welcome bruh.|||When I get in key with my emotions - I go quite ballistic. I'm not sure if it's just hormones or what": "description"})

In [ ]:
dff

,type,posts
0,INFJ,'http://www.youtube.com/watch?v=qsXHcwe3krw|||http://41.media.tumblr.com/tumblr_lfouy03PMA1qa1rooo1_500.jpg|||enfp and intj moments https://www.youtube.com/watch?v=iz7lE1g4XM4 sportscenter not top ten plays https://www.youtube.com/watch?v=uCdfze1etec pranks|||What has been the most life-changing experience in your life?|||http://www.youtube.com/watch?v=vXZeYwwRDw8 http://www.youtube.com/watch?v=u8ejam5DP3E On repeat for most of today.|||May the PerC Experience immerse you.|||The last ...
1,ENTP,'I'm finding the lack of me in these posts very alarming.|||Sex can be boring if it's in the same position often. For example me and my girlfriend are currently in an environment where we have to creatively use cowgirl and missionary. There isn't enough...|||Giving new meaning to 'Game' theory.|||Hello *ENTP Grin* That's all it takes. Than we converse and they do most of the flirting while I acknowledge their presence and return their words with smooth wordplay and more cheeky grins.|||This...
2,INTP,"'Good one _____ https://www.youtube.com/watch?v=fHiGbolFFGw|||Of course, to which I say I know; that's my blessing and my curse.|||Does being absolutely positive that you and your best friend could be an amazing couple count? If so, than yes. Or it's more I could be madly in love in case I reconciled my feelings (which at...|||No, I didn't; thank you for a link!|||So-called Ti-Si loop (and it can stem from any current topic/obsession) can be deadly. It's like when you're stuck in your o..."
3,INTJ,"'Dear INTP, I enjoyed our conversation the other day. Esoteric gabbing about the nature of the universe and the idea that every rule and social code being arbitrary constructs created...|||Dear ENTJ sub, Long time no see. Sincerely, Alpha|||None of them. All other types hurt in deep existential ways that I want no part of.|||Probably a sliding scale that depends on individual preferences, like everything in humanity.|||Draco Malfoy also. I'd say he's either 358 or 368.|||I'm either 358..."
4,ENTJ,'You're fired.|||That's another silly misconception. That approaching is logically is going to be the key to unlocking whatever it is you think you are entitled to. Nobody wants to be approached with BS...|||But guys... he REALLY wants to go on a super-duper-long-ass vacation. C'mon guys. His boss just doesn't listen or get it. He even approached him logically and everything.|||Never mind. Just go on permanent vacation.|||Two months? I wouldn't be crazy about the idea. If you are really hi...
...,...,...
8670,ISFP,"'https://www.youtube.com/watch?v=t8edHB_h908|||IxFP just because I always think of cats as Fi doms for some reason. https://www.youtube.com/watch?v=ib28iGc9go4|||Especially on websites that have become a haven for neo nazis, such as PerC.|||I'm such a nerd. I've been learning Dutch on Duolingo and I'm having so much fun :)|||Duolingo is the shit! Oh my god I love this XD|||Right wingers do this because they lack political consciousness.|||I doubt it, but if they are real then I hope to he..."
8671,ENFP,"'So...if this thread already exists someplace else (which it does: http://personalitycafe.com/enfp-forum-inspirers/116902-movies-loved-enfp.html#post2952851), how the heck to I delete this one here???|||Ooops...I guess I didn't look too hard because I just started a Movies every ENFP should watch thread...|||So, this came up in the You Know You're An ENFP When... thread. (@Pucca : here we go! IT IS DONE :D) This isn't a list of films with ENFPs IN them, but rather ones that you believe E..."
8672,INTP,'So many questions when i do these things. I would take the purple pill. Pick winning lottery numbers. Do whatever I want.|||I worked in a brewery for a couple years here. I dont know if you're familiar with the local micros/craft at all but Mill st. was the name of it. Their tankhouse is my favorite of anything local.|||I am going to summon the great and powerful mr cafebot and his song of destruction. The four horseman ride. 

In [ ]:
df=dff

### Data splitting

In [ ]:
X_train, X_test = train_test_split(df, test_size=.2, stratify=df['type'])

### EDA on training data

- check NaNs

In [ ]:
X_train.isna().sum()

type     0
posts    0
dtype: int64

- check duplicates

In [ ]:
X_train.duplicated().sum()

0

- show a representative sample of data texts to find out required preprocessing steps

In [ ]:
X_train.sample(10)

,type,posts
187,INTP,So who is the Rush Limbaugh of the intuitives and the Uncle Ruckus of the Sensors?|||A few weeks and some months not fafe to face social interaction.|||Introverted thinking and extroverted intuition give the ability to pattern seek and logically find a way and correlate information and throw away old information. In reality just make us great at...|||INTPs are the smartest and nowhere getting around it. We suck at social levels until we get older and than we can control the social setting wi...
6126,INFP,"'I frolic even when people are around|||You're now chatting with a random stranger. Say hi! Official messages from Omegle will not be sent with the label 'Stranger:'. Strangers claiming to represent Omegle are lying. You: hi Stranger:...|||Don't think, just do|||Oh, i got ya, ya now that I read it over structure does not exist much in it. I am incredibly a mess when trying to explain things, no structure what so ever. So do INFP's tend to have no structure...|||I am not sure what you mean. B..."
696,INTP,"'Yes I really have thought a little bit about this too, the poor intx espeically intj, that should be societies next generetaion intellectuals and all that stuff. There is quite a staggering amount of...|||14/15 I hate that bottom left fucker on the korealike flag on question 14. I did not get that one ><' EDIT: Oh I get it now, it moves counterclockwise 45degreees then 90 then 180 so on ? it...|||Hehe, yep he has grand delusional vision of what he thinks is good and bad, he thinks it is g..."
3016,INFP,"'I'm currently broadcasting. If anyone is curious, shoot me a PM and I'll give you a link.|||http://www.youtube.com/watch?v=hSo-ttyk31I|||Heh, I'm going to have to admit that I'm a little disappointed in the guarded/muted responses in this thread. I know you guys are freakier than I like x body part or i like x perfect...|||How have I not come across this thread yet? This is totally my element. First thing everyone needs to understand is that sexuality is about as normal as people are. ..."
5432,ESTP,"'Pro-life|||I am married to a major feeler like yourself, he's ESFJ. He says the exact things you said about your SO. We are difficult to get close to and we are very slow moving with sharing our inner personal...|||I'm not one to just sit and talk, it gets boring after a while. To be honest most friendships I enjoy with other girls are around doing things together.|||She could be an ESTP. I was never shy with adults as a kid but that could be more nurture than nature. I can also see how ..."
5694,ENFJ,"'Sorry, but this is, to put it quite frankly, bullshit. It's victim blaming at its finest. The logic here is akin to She was raped because she wore a short skirt. Almost any psychologist will...|||I've never cheated and wouldn't even dream of it, but I have been cheated on by at least two ex-girlfriends and one ex-wife. I really can't speak as to why one of the ex-girlfriends cheated, but...|||Congratulations Seeker, I definitely wish you the best. I'm at the 6 month mark with my I..."
2197,INFJ,"'4w5. I am the opposite of aggressive.|||Another thought: If INFJs do tend to like people who are hard to get, it might have something to do with the whole perfectionist-about-romance thing. I know I've always had a disconnect between the...|||It's funny that everyone seems to be giving examples of shy people as hard to get. For me, the extroverts I get crushes on are the ones I really give up on. It's easier for me to become close to an...|||Quang has it spot-on. As a 4(w5), I've found mys..."
1782,INTJ,"'I've not really seen any sensor-haters here so far... But maybe I've missed them, and I haven't been here for very long. Seen a lot of posts about sensor-haters though, that's true. As for...|||My friends used to get really annoyed with me in high school. Whenever they'd start gossiping, I'd make my opinion clear that I don't care and I didn't want to hear it. It was mostly about people I...|||I don't know if it w

- check dataset balancing

In [ ]:
X_train['type'].value_counts()

type
INFP    1466
INFJ    1176
INTP    1043
INTJ     873
ENTP     548
ENFP     540
ISTP     270
ISFP     217
ENTJ     185
ISTJ     164
ENFJ     152
ISFJ     133
ESTP      71
ESFP      38
ESFJ      33
ESTJ      31
Name: count, dtype: int64

### Our data not balanced , sure we will see the same proportion in the test


- Cleaning and Preprocessing steps:
    - 1. The `preprocess_text` function contains the following preprocessing steps:
        - Remove mentions and hashtags.
        - Remove URLs.
        - Remove emojis, symbols, and non-ASCII characters.
        - Lowercase the text.
        - Handle contractions using `contractions.fix()`.
        - Remove punctuation using `text.translate(str.maketrans('', '', string.punctuation))`.
        - Remove digits using `re.sub(r'\d+', '', text)`.
        - Remove whitespace using `re.sub(r'\s+', ' ', text).strip()`.
    - 2. The `stem_text` function applies stemming to the text using the Porter Stemmer from NLTK:
        - Tokenize the text into words.
        - Apply stemming to each word.
        - Join the stemmed words back into a single string.



### Cleaning and Preprocessing

In [ ]:
def preprocess_text(text):
    # Remove mentions and hashtags
    text = re.sub(r'@[A-Za-z0-9_]+|#[A-Za-z0-9_]+', '', text)
    # Remove URLs
    text = re.sub(r'http\S+', '', text)
    # Remove emojis, symbols, non-ASCII characters
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    # Lowercasing
    text = text.lower()
    # Handle Contractions (Wee need it before remove punctuation)
    text = contractions.fix(text)
    # Remove Punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))
    # Remove digits
    text = re.sub(r'\d+', '', text)
    # Remove Whitspace like (  jkl   klkl      fgh  )
    text=re.sub(r'\s+', ' ', text).strip()

    return text


In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
from nltk.stem import PorterStemmer
#stemmer = PorterStemmer()

In [ ]:
def stem_text(text):
    # init Porter Stemmer
    stemmer = PorterStemmer()
    # Tokenize into wordss
    words = text.split()
    # stemming to each word
    stemmed_words = [stemmer.stem(word) for word in words]
    # Join the stemmed words back into === > a single string
    stemmed_text = ' '.join(stemmed_words)
    return stemmed_text

## We need to test our preprocessing before but it in the pipline

In [ ]:
dff = df.iloc[:,1].apply(preprocess_text)

In [ ]:
dff=dff.apply(stem_text)
dff

0       and intj moment sportscent not top ten play prankswhat ha been the most lifechang experi in your life on repeat for most of todaymay the perc experi immers youth last thing my infj friend post on hi facebook befor commit suicid the next day rest in peac enfj sorri to hear of your distress it is onli natur for a relationship to not be perfect all the time in everi moment of exist tri to figur the hard time as time of growth as welcom and stuff game set matchprozac wellbrutin at least thirti m...
1       i am find the lack of me in these post veri alarmingsex can be bore if it is in the same posit often for exampl me and my girlfriend are current in an environ where we have to creativ use cowgirl and missionari there is not enoughgiv new mean to game theoryhello entp grin that is all it take than we convers and they do most of the flirt while i acknowledg their presenc and return their word with smooth wordplay and more cheeki grinsthi lack of balanc and hand eye coordinationr iq

## Before Preprocessing

In [ ]:
df.sample(10,random_state=42)

,type,posts
2802,INTP,"'This. When I lie it's to avoid an unreasonable response for something I did that was completely justifiable to me but if I was to try to justify to someone else they'd be too hung up on what I did. I...|||Your IQ (SD15): 128|||I typically have very difficult times maintaining friends I've come to realize. I've had people I talk to for years and I still don't want to talk to them... I DID, but then I just got bored of them,...|||An INTJ friend of mine told me she really hates eating because ..."
2166,INTJ,"'I said zero flexibility and little time for dating. Going around sifting through dates to find one that clicks isn't going to be as viable. That has nothing to do with maintaining a relationship I...|||How you've come to the conclusion I have nothing to offer merely because I am busy is beyond me. No not really. Again, more made up stuff.|||okay, this isn't cool. you're specifically looking to get into a relationship because you're about to enter a life phase where you will not hav..."
1919,INTP,"'This has to be written with bias or something, I'm not sure how you could even say that truthfully. For some reason you put subjectivity for both God and man? It says nothing. That makes no...|||??? How? The main priority of the church is written throughout it's texts for the past nearly 2000 years and reverberated throughout the church's dialogues both before and after the incident about...|||That's an annoying phrasing, as it's just obtuse and confusing to newcomers. I don't blame you..."
360,ENFP,"'HAuhuHAuh You might be right Muhicz. I'm just wondering if these steps come exactly the same ages like it said, it's tought to believe scientifically... but, it doesn't really matter, the point is...|||Thank you, MisterJordan. Your words are helpfull. It helps me to think that the conflicting beetwen Fi and Ti could be an inner development of myself, improving who I am. But I must say I'm not the...|||I've read about the Fe and Fi functions, I'm a Fi. I also know well about the MBTI types, ..."
1115,ENTJ,"'Her parents are kind of the go to college = good job people, and they want her to have a stable career. She on the other hand, wants to travel, perhaps teach English in foreign lands (especially in...|||I have a friend who is going through this crisis. She is about to graduate high school, and has absolutely know idea what to do with her life. She doesn't have anything she's really passionate...|||I recently met a few waitresses at one restaurant in my town who served for a living. I know..."
623,INFJ,"'that's so ironic, considering how your signature is cats are the better people.. cats are known to kill or play with their prey for fun. you think that's wrong because that's how you were...|||ooh, I'm a virgo rising INFJ! virgo is also one of my fav zodiac signs. INFJ's is that kind and honest childhood friend who even after years of not talking would be happy to check up on you and...|||I'm consumed by machiavellianism in dark periods of my life. I sometimes just get fed up by all the fa..."
1741,INTJ,"'Well said , Codger.|||You will like it. :proud:|||Never mind. Apologies for the intrusion.|||True. Hey Cetanu. Been a while. How ya doin ?|||Hey ChancyRose, how are ya ? Add me to that list too. The punching bag thing didn't really work...|||I've got mine on; and the world is still retarded....among other things.|||Good luck.|||u200b Good morning people..|||Anyway... Nothing to do but go to bed and deal with whatever bullshit my subconscious has in store for me. At least it would be a chan..."
5818,ENTP,"'Just curious. I realise this is a bit of a stupid question, and bound to generalisations, but would there be anything the way NFs look, dress etc which is distinguishable from others?|||I can relate to this so much it's ridiculous|||Nothing goes unnoticed.|||I 'deal' with loneliness by: - Rationalising and shooing it away (only works less than half of the time) - Focusing on my main current goal (successfull

## After Preprocessing

In [ ]:
dff.sample(10,random_state=42)

2802    thi when i lie it is to avoid an unreason respons for someth i did that wa complet justifi to me but if i wa to tri to justifi to someon els they would be too hung up on what i did iyour iq sd i typic have veri difficult time maintain friend i have come to realiz i have had peopl i talk to for year and i still do not want to talk to them i did but then i just got bore of theman intj friend of mine told me she realli hate eat becaus it is so repetit as an intp i do not realli feel bother i do...
2166    i said zero flexibl and littl time for date go around sift through date to find one that click is not go to be as viabl that ha noth to do with maintain a relationship ihow you have come to the conclus i have noth to offer mere becaus i am busi is beyond me no not realli again more made up stuffokay thi is not cool you are specif look to get into a relationship becaus you are about to enter a life phase where you will not have ani resourc to give to a relationship that isi do tak

## Now we will put them together in custum tansformer with pipline :


#### Extra: use custom scikit-learn Transformers

Using custom transformers in scikit-learn provides flexibility, reusability, and control over the data transformation process, allowing you to seamlessly integrate with scikit-learn's pipelines, enabling you to combine multiple preprocessing steps and modeling into a single workflow. This makes your code more modular, readable, and easier to maintain.

##### link: https://www.andrewvillazon.com/custom-scikit-learn-transformers/

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

class CustomTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        # self.parameter1 = parameter1
        # self.parameter2 = parameter2
        pass

        # Add any initialization code here

    def fit(self, X, y=None):
        # Add code for fitting the transformer here
        return df

    def transform(self, X):
        # Add code for transforming the data here
        transformed_X = X.copy()  # Example: Just copying the data
        transformed_X=transformed_X.iloc[:,1].apply(preprocess_text)
        transformed_X=transformed_X.apply(stem_text)
        return transformed_X

    def fit_transform(self, X, y=None):
        # This function combines fit and transform
        self.fit(X, y)
        return self.transform(X)

In [ ]:
# Init CustomTransformer
custom_transformer = CustomTransformer()

# Fit and transform X_train
transformed_X_train = custom_transformer.fit_transform(X_train)


In [ ]:
transformed_X_train

6345    i have been told i have a death glare my wife while she is use to me and ha no fear of me fear for other when i use it i do not even know when i use itthi you are an istp right most of us are fairli plainspoken folksoh yeah that is me got to check you out from a distanc and see what you are all about befor i consid let you in the inner circl onc you are in you are in for life with the except ofi think istp would fare well for that hour period do not expect that most of us would be out do stu...
8391    inconsider is a word i see thrown around a lot on thi forum as well as gener describ men and use he pronoun or impli male and to touch on the inconsider part i think it would bewhen i wa littl i want to be an archaeologist then a marin biologist then a detect and now a writer i actual still want to be all these thingsi guess i must be extrem picki i meet peopl and i know just know they are not it they may be for someon els but not for me and at the moment i think i have a lot of 

In [ ]:
# Transform X_test
transformed_X_test = custom_transformer.transform(X_test)

In [ ]:
transformed_X_test

806     enfj theodor twombl infp ami estprefer entp archetyp richard feynman ergo sum your exist is not depend on other human be or even your environ percept of your exist your exist depend on your own thought and action so next timefear itself sinc one ideal of what make a human adult male manli is so wide subject one specif mbti type cannot be assign the titl have establish that my idea of manli is someon likei am atheist when it come to the god of the variou man made religion as we have discov th...
6142    when wa your birthday you should watch it as soon as possibl then how wa ameliesuck punch it ha some pretti awesom actionfight scenesth color purpl by alic walker one flew over the cuckoo nest by ken kesey a clockwork orang by anthoni burgress i have not read it yet but i hear it is goodbut ha it is own slanglanguag catcher in the rye jd saling the great gatsbi f scott fitzgerald memoir of a geisha arthur golden the color purpl alic walkeri read quit a bit but not a lot of theth 

In [ ]:
transformed_X_train.shape , transformed_X_test.shape

((6940,), (1735,))

In [ ]:
# Check again if there is any duplication after preprocessing
transformed_X_train.duplicated().sum()

0

In [ ]:
# import numpy as np
# from collections import Counter
# from keras.preprocessing.sequence import pad_sequences

# def process_text_data(X_train):
#     # Step 1: Tokenize the data
#     word_counts = Counter()
#     for text in X_train:
#         word_counts.update(text.split())

#     # Create word-to-index and index-to-word mappings
#     word2idx = {word: idx + 1 for idx, (word, _) in enumerate(word_counts.most_common())}
#     idx2word = {idx: word for word, idx in word2idx.items()}

#     # Step 2: Convert text data into sequences of integers
#     max_seq_length = max(len(text.split()) for text in X_train)
#     sequences = []
#     for text in X_train:
#         sequence = [word2idx[word] for word in text.split()]
#         sequences.append(sequence)

#     # Step 3: Pad sequences (if necessary)
#     padded_sequences = pad_sequences(sequences, maxlen=max_seq_length)

#     return padded_sequences, word2idx, idx2word


### `process_text_data` Function Summary

This function processes the text data for input to a neural network model.

#### Parameters:
- `transformed_X_train` (list): A list of strings, where each string represents a document or sentence.

#### Returns:
- `padded_sequences` (numpy.ndarray): A 2D numpy array representing the padded sequences of word indices.
- `word2idx` (dict): A dictionary mapping words to their corresponding indices.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from keras.preprocessing.sequence import pad_sequences

def process_text_data(transformed_X_train):
    # strings ==> tokens
    tokenized_X_train = [doc.split() for doc in transformed_X_train]

    # CountVectorizer ==> try to extract unique values
    vectorizer = CountVectorizer()

    # Join tokenized sentences ==> strings
    transformed_X_train_strings = [' '.join(tokens) for tokens in tokenized_X_train]

    # Fit on data ==> extract unique words
    vectorizer.fit(transformed_X_train_strings)

    # unique words and their ==> indices
    word2idx = {word: idx + 1 for idx, word in enumerate(vectorizer.get_feature_names_out())}
    # index 0 to unknown words
    word2idx['<UNK>'] = 0

    # sentences ==> sequences of word indices
    sequences = []
    for tokens in tokenized_X_train:
        word_indices = [word2idx.get(word, 0) for word in tokens]
        sequences.append(word_indices)

    # Paddingg
    max_seq_length = max(len(seq) for seq in sequences)
    padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

    return padded_sequences, word2idx,max_seq_length


In [ ]:
padded_sequences, word2idx,max_seq_length = process_text_data(transformed_X_train)

In [ ]:
padded_sequences[0].shape,padded_sequences[1].shape

((1910,), (1910,))

In [ ]:
padded_sequences[0],padded_sequences[1]

(array([    0, 80134, 19073, ...,     0,     0,     0], dtype=int32),
 array([90342, 96220,     0, ...,     0,     0,     0], dtype=int32))

In [ ]:
max_seq_length

1910

## Applay on the test data :-

In [ ]:
def process_text_data(transformed_data, word2idx, max_seq_length):
    """
    Processes the text data for input to a neural network model.

    Args:
    transformed_data (list): A list of strings, where each string represents a document or sentence.
    word2idx (dict): A dictionary mapping words to their corresponding indices.
    max_seq_length (int): Maximum sequence length.

    Returns:
    numpy.ndarray: A 2D numpy array representing the padded sequences of word indices.
    """
    # strings ==> tokens
    tokenized_data = [doc.split() for doc in transformed_data]

    # sentencee ==> to sequences of word indices
    sequences = []
    for tokens in tokenized_data:
        word_indices = [word2idx.get(word, 0) for word in tokens]
        sequences.append(word_indices)

    # Padding
    padded_sequences = pad_sequences(sequences, maxlen=max_seq_length, padding='post')

    return padded_sequences


In [ ]:
padded_sequences_test = process_text_data(transformed_X_test, word2idx, max_seq_length=padded_sequences.shape[1])

In [ ]:
padded_sequences_test[0].shape

(1910,)

In [ ]:
## In case you have multi-labeles
## We need to decode y here
from sklearn.preprocessing import LabelEncoder

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Encode the string labels into integers
y_train_encoded = label_encoder.fit_transform(X_train['type'])
y_test_encoded = label_encoder.transform(X_test['type'])

# Convert the integer labels to one-hot encoded format
y_train_categorical = tf.keras.utils.to_categorical(y_train_encoded)
y_test_categorical = tf.keras.utils.to_categorical(y_test_encoded)


In [ ]:
y_test_encoded

array([ 3, 12,  2, ..., 10,  8, 10])

In [ ]:
y_train_categorical

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

**You  are doing Great so far!**

### Modelling

In [ ]:
VOCAB_SIZE = len(word2idx)
VOCAB_SIZE

221387

In [ ]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Embedding(VOCAB_SIZE, 128),
#     tf.keras.layers.SimpleRNN(64),
#     tf.keras.layers.Dense(64, activation='relu'),
#     tf.keras.layers.Dense(16, activation='softmax')
# ])

In [ ]:
# model.compile(loss=tf.keras.losses.CategoricalCrossentropy,
#               optimizer=tf.keras.optimizers.Adam(),
#               metrics=['accuracy'])

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 128),
    tf.keras.layers.SimpleRNN(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(16, activation='softmax')
])

model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

history = model.fit(padded_sequences, y_train_categorical, epochs=5, batch_size=128,
                    validation_data=(padded_sequences_test, y_test_categorical),
                    validation_steps=30)


Epoch 1/5
55/55 [==============================] - ETA: 0s - loss: 2.3986 - accuracy: 0.1833

55/55 [==============================] - 114s 2s/step - loss: 2.3986 - accuracy: 0.1833 - val_loss: 2.2866 - val_accuracy: 0.2110
Epoch 2/5
55/55 [==============================] - 109s 2s/step - loss: 2.2878 - accuracy: 0.2112
Epoch 3/5
55/55 [==============================] - 104s 2s/step - loss: 2.2875 - accuracy: 0.2013
Epoch 4/5
55/55 [==============================] - 112s 2s/step - loss: 2.2854 - accuracy: 0.2078
Epoch 5/5
55/55 [==============================] - 104s 2s/step - loss: 2.2838 - accuracy: 0.2112


#### Evaluation

**Evaluation metric:**
Precision

In [ ]:
# predictions on the test data
y_pred = model.predict(padded_sequences_test)
# probabilities to class labels
y_pred_labels = np.argmax(y_pred, axis=1)

55/55 [==============================] - 8s 147ms/step


In [ ]:
# pd.DataFrame(y_pred_labels).value_counts()

In [ ]:
def calculate_precision(model, test_data, true_labels):
    """
    Calculate precision for the given model on the test data.

    Args:
    model: The trained model.
    test_data: The test data (padded sequences).
    true_labels: The true labels for the test data.

    Returns:
    float: The precision score.
    """
    # Make predictions on the test data
    y_pred = model.predict(test_data)

    # Convert probabilities to class labels
    y_pred_labels = np.argmax(y_pred, axis=1)

    # Calculate precision
    precision = precision_score(true_labels, y_pred_labels, average='micro')

    return precision


In [ ]:
precision = calculate_precision(model, padded_sequences_test, y_test_encoded)
print("Precision:", precision)

55/55 [==============================] - 8s 139ms/step
Precision: 0.21095100864553315


### Enhancement : will used more complex RNN

Can we make it work in both directions ?

In [ ]:
model_bi = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 128),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.SimpleRNN(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(16, activation='softmax')
])

In [ ]:
model_bi.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history = model_bi.fit(padded_sequences, y_train_categorical, epochs=5, batch_size=128,
                    validation_data=(padded_sequences_test, y_test_categorical),
                    validation_steps=30)

Epoch 1/5
55/55 [==============================] - ETA: 0s - loss: 2.3307 - accuracy: 0.1927

55/55 [==============================] - 399s 7s/step - loss: 2.3307 - accuracy: 0.1927 - val_loss: 2.2937 - val_accuracy: 0.1689
Epoch 2/5
55/55 [==============================] - 387s 7s/step - loss: 2.2477 - accuracy: 0.2310
Epoch 3/5
55/55 [==============================] - 388s 7s/step - loss: 1.9955 - accuracy: 0.3765
Epoch 4/5
55/55 [==============================] - 394s 7s/step - loss: 1.4019 - accuracy: 0.5733
Epoch 5/5
55/55 [==============================] - 389s 7s/step - loss: 0.6188 - accuracy: 0.8261


In [ ]:
precision = calculate_precision(model_bi, padded_sequences_test, y_test_encoded)
print("Precision:", precision)

55/55 [==============================] - 30s 540ms/step
Precision: 0.13659942363112393


## Lets try onther one :

### Implementing GRU

In [ ]:
gru_bi = tf.keras.models.Sequential([
    tf.keras.layers.Embedding(VOCAB_SIZE, 256),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64, return_sequences=True)),
    tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(16, activation='softmax')
])

In [ ]:
gru_bi.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])

In [ ]:
history = gru_bi.fit(padded_sequences, y_train_categorical, epochs=10, batch_size=128,
                    validation_data=(padded_sequences_test, y_test_categorical),
                    validation_steps=30)

Epoch 1/10
55/55 [==============================] - ETA: 0s - loss: 2.3776 - accuracy: 0.1978

55/55 [==============================] - 30s 422ms/step - loss: 2.3776 - accuracy: 0.1978 - val_loss: 2.2858 - val_accuracy: 0.2110
Epoch 2/10
55/55 [==============================] - 20s 360ms/step - loss: 2.2475 - accuracy: 0.2216
Epoch 3/10
55/55 [==============================] - 20s 356ms/step - loss: 2.0404 - accuracy: 0.3043
Epoch 4/10
55/55 [==============================] - 20s 355ms/step - loss: 1.6685 - accuracy: 0.4465
Epoch 5/10
55/55 [==============================] - 20s 356ms/step - loss: 1.2650 - accuracy: 0.5849
Epoch 6/10
55/55 [==============================] - 19s 341ms/step - loss: 0.9241 - accuracy: 0.7082
Epoch 7/10
55/55 [==============================] - 19s 343ms/step - loss: 0.6588 - accuracy: 0.7937
Epoch 8/10
55/55 [==============================] - 19s 353ms/step - loss: 0.4472 - accuracy: 0.8604
Epoch 9/10
55/55 [==============================] - 19s 355ms/step - loss: 0.3226 - accuracy: 0.9000
Epoch 10/10
55/55 [==============================] - 19s 340

In [ ]:
precision = calculate_precision(gru_bi, padded_sequences_test, y_test_encoded)
print("Precision:", precision)

55/55 [==============================] - 6s 93ms/step
Precision: 0.1723342939481268


## Lets try onther one :

In [ ]:
from tensorflow.keras.layers import Embedding, Bidirectional, GRU, Dense, Dropout

# Define the model
gru_bi_complex = tf.keras.models.Sequential([
    Embedding(VOCAB_SIZE, 256),
    Bidirectional(GRU(128, return_sequences=True)),
    Dropout(0.2),
    Bidirectional(GRU(128, return_sequences=True)),
    Bidirectional(GRU(128)),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compile the model
gru_bi_complex.compile(loss='categorical_crossentropy',
                       optimizer='adam',
                       metrics=['accuracy'])



In [ ]:
history = gru_bi_complex.fit(padded_sequences, y_train_categorical, epochs=10, batch_size=128,
                    validation_data=(padded_sequences_test, y_test_categorical),
                    validation_steps=30)

Epoch 1/10
55/55 [==============================] - ETA: 0s - loss: 0.3989 - accuracy: 0.8726

55/55 [==============================] - 48s 864ms/step - loss: 0.3989 - accuracy: 0.8726 - val_loss: 5.5954 - val_accuracy: 0.1712
Epoch 2/10
55/55 [==============================] - 43s 778ms/step - loss: 0.3814 - accuracy: 0.8821
Epoch 3/10
55/55 [==============================] - 43s 777ms/step - loss: 0.2834 - accuracy: 0.9170
Epoch 4/10
55/55 [==============================] - 43s 784ms/step - loss: 0.2486 - accuracy: 0.9235
Epoch 5/10
55/55 [==============================] - 43s 789ms/step - loss: 0.2150 - accuracy: 0.9336
Epoch 6/10
55/55 [==============================] - 43s 781ms/step - loss: 0.1856 - accuracy: 0.9414
Epoch 7/10
55/55 [==============================] - 43s 775ms/step - loss: 0.1421 - accuracy: 0.9561
Epoch 8/10
55/55 [==============================] - 43s 782ms/step - loss: 0.1380 - accuracy: 0.9572
Epoch 9/10
55/55 [==============================] - 43s 778ms/step - loss: 0.1199 - accuracy: 0.9615
Epoch 10/10
55/55 [==============================] - 43s 780

In [ ]:
precision = calculate_precision(gru_bi_complex, padded_sequences_test, y_test_encoded)
print("Precision:", precision)

55/55 [==============================] - 9s 163ms/step
Precision: 0.1729106628242075


## Lets try onther one :

In [ ]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout

# Define the model
model_complex = tf.keras.models.Sequential([
    Embedding(VOCAB_SIZE, 128),
    SimpleRNN(1024, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(512, return_sequences=True),
    Dropout(0.2),
    SimpleRNN(256),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dense(16, activation='softmax')
])

# Compile the model
model_complex.compile(loss='categorical_crossentropy',
                      optimizer='adam',
                      metrics=['accuracy'])


In [ ]:
history = model_complex.fit(padded_sequences, y_train_categorical, epochs=10, batch_size=128,
                    validation_data=(padded_sequences_test, y_test_categorical),
                    validation_steps=30)

Epoch 1/10
55/55 [==============================] - ETA: 0s - loss: 2.4375 - accuracy: 0.1781

55/55 [==============================] - 353s 6s/step - loss: 2.4375 - accuracy: 0.1781 - val_loss: 2.3014 - val_accuracy: 0.2110
Epoch 2/10
55/55 [==============================] - 337s 6s/step - loss: 2.3418 - accuracy: 0.1849
Epoch 3/10
55/55 [==============================] - 336s 6s/step - loss: 2.3216 - accuracy: 0.1957
Epoch 4/10
55/55 [==============================] - 344s 6s/step - loss: 2.3204 - accuracy: 0.1938
Epoch 5/10
55/55 [==============================] - 344s 6s/step - loss: 2.3159 - accuracy: 0.1986
Epoch 6/10
55/55 [==============================] - 337s 6s/step - loss: 2.3150 - accuracy: 0.2037
Epoch 7/10
10/55 [====>.........................] - ETA: 4:32 - loss: 2.3495 - accuracy: 0.1883

### Conclusion and final results


In [ ]:
precision = calculate_precision(model_complex, padded_sequences_test, y_test_encoded)
print("Precision:", precision)

55/55 [==============================] - 8s 139ms/step
Precision: 0.21095100864553315


#### Done!